In [1]:
!pip install keras
!pip install tqdm

Exception:
Traceback (most recent call last):
  File "C:\Users\Max2x97\Anaconda3\lib\site-packages\pip\basecommand.py", line 215, in main
    status = self.run(options, args)
  File "C:\Users\Max2x97\Anaconda3\lib\site-packages\pip\commands\install.py", line 335, in run
    wb.build(autobuilding=True)
  File "C:\Users\Max2x97\Anaconda3\lib\site-packages\pip\wheel.py", line 749, in build
    self.requirement_set.prepare_files(self.finder)
  File "C:\Users\Max2x97\Anaconda3\lib\site-packages\pip\req\req_set.py", line 380, in prepare_files
    ignore_dependencies=self.ignore_dependencies))
  File "C:\Users\Max2x97\Anaconda3\lib\site-packages\pip\req\req_set.py", line 554, in _prepare_file
    require_hashes
  File "C:\Users\Max2x97\Anaconda3\lib\site-packages\pip\req\req_install.py", line 278, in populate_link
    self.link = finder.find_requirement(self, upgrade)
  File "C:\Users\Max2x97\Anaconda3\lib\site-packages\pip\index.py", line 465, in find_requirement
    all_candidates = self.fi

In [9]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras import regularizers
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.initializers import glorot_normal
#import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#from kt_utils import *

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

In [3]:
import pickle
pickle_off = open("dataX_final.pickle","rb")
X = pickle.load(pickle_off)
print(X.shape)
pickle_on = open("dataY_final.pickle","rb")
Y = pickle.load(pickle_on)
print(Y.shape)
p = np.random.permutation(X.shape[0])
X = X[p]
Y = Y[p]
X_test_orig = X[2698:]
X_train_orig = X[:2698]
Y_test_orig = Y[2698:]
Y_train_orig = Y[:2698]
print(X_test_orig.shape, X_train_orig.shape)

(2997, 224, 224, 3)
(2997, 12)
(299, 224, 224, 3) (2698, 224, 224, 3)


In [4]:
#X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset_new()
X_train = X_train_orig
X_test = X_test_orig
# Normalize image vectors
#X_train = X_train_orig/255.
#X_test = X_test_orig/255.
x = 0
for i in X_train:
    i = i/255
    x +=1
    if x%100 == 0:
        print(x)
for i in X_test:
    i = i/255

Y_train = Y_train_orig
Y_test = Y_test_orig

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
number of training examples = 2698
number of test examples = 299
X_train shape: (2698, 224, 224, 3)
Y_train shape: (2698, 12)
X_test shape: (299, 224, 224, 3)
Y_test shape: (299, 12)


In [10]:
def base_model(input_shape,classes = 12):
    """Implementation of the BaseModel.
       Arguments: input_shape -- shape of the images of the dataset
       Returns: model -- a Model() instance in Keras
    """
    X_input = Input(input_shape)
    
    #padding of 1
    X = ZeroPadding2D(padding=(1, 1))
    
    # conv layer no padding 3x3  32 filters stride=1
    X = Conv2D(32,(3,3),strides=(2,2),name='conv0')(X_input)
    X = BatchNormalization(axis = 3, name='bn0')(X)
    X = Activation('relu')(X)
    
    #pooling layer(max 2x2 stride 2)
    X = MaxPooling2D((2,2),name='maxpool0')(X)
    
    #padding of 1
    X = ZeroPadding2D(padding = (1,1))
    
    # conv layer no padding 3x3  32 filters stride=1
    X = Conv2D(32,(3,3),strides=(2,2),name='conv1')(X_input)
    X = BatchNormalization(axis = 3, name='bn1')(X)
    X = Activation('relu')(X)
    
    #pooling layer(max 2x2 stride 2)
    X = MaxPooling2D((2,2),name='maxpool1')(X)
    
    #flatten()
    X = Flatten()(X)
    X = Dense(228,activation='relu',activity_regularizer= regularizers.l2(0.01) ,name='fc1', kernel_initializer = glorot_normal(seed=None))(X)
    X = Dense(classes,activation='softmax',activity_regularizer= regularizers.l2(0.01) ,name='fc2')(X)
    
    model = Model(inputs = X_input, outputs = X, name='base_model')
    ### END CODE HERE ###
    
    return model
    

In [11]:
bsm = base_model((224,224,3))

In [12]:
bsm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
bsm.fit(x=X_train,y=Y_train,epochs=20,batch_size=64)


Epoch 1/20
2698/2698 [==============================] - 153s 57ms/step - loss: 4.2753 - acc: 0.1186
Epoch 2/20
2698/2698 [==============================] - 148s 55ms/step - loss: 2.5199 - acc: 0.1197
Epoch 3/20
2698/2698 [==============================] - 174s 64ms/step - loss: 2.5093 - acc: 0.1197
Epoch 4/20
2698/2698 [==============================] - 182s 67ms/step - loss: 2.5002 - acc: 0.1197
Epoch 5/20
2698/2698 [==============================] - 184s 68ms/step - loss: 2.4922 - acc: 0.1197
Epoch 6/20
2698/2698 [==============================] - 177s 66ms/step - loss: 2.4852 - acc: 0.1201
Epoch 7/20
2698/2698 [==============================] - 183s 68ms/step - loss: 2.4792 - acc: 0.1223
Epoch 8/20
2698/2698 [==============================] - 161s 60ms/step - loss: 2.4738 - acc: 0.1223
Epoch 9/20
2698/2698 [==============================] - 158s 59ms/step - loss: 2.4692 - acc: 0.1223
Epoch 10/20
2698/2698 [==============================] - 147s 55ms/step - loss: 2.4651 - acc: 0.1223

In [ ]:
bsm.save("my_model_RELU")

In [ ]:
bsm.evalulate(x=X_test,y=Y_test, batch_size = 64)

In [14]:
from sklearn import datasets, svm, metrics
classifier = svm.SVC(gamma=0.001)

In [26]:
type(X_train)
twoX_train.shape

(406124544,)

In [35]:
twoX_train =[]
twoY_train =[]
for i in X_train:
    twoX_train.append(i.flatten())
for i in Y_train:
    twoY_train.append(i.flatten())
    
(twoX_train[1].shape)

(150528,)

In [ ]:
classifier.fit(twoX_train,twoY_train)